In [ ]:
import pandas as pd
import pyarrow as pa

In [ ]:
import pandas as pd
import pyarrow as pa

print("Reading arrow file...")
mmap = pa.memory_map('/content/drive/MyDrive/Amazon ML Challenge/amazon_ml_challenge_cleaned_dataset/amazon_ml_challenge_cleaned_dataset/train/data-00000-of-00001.arrow')


Reading arrow file...


In [ ]:
from pyarrow import json
fn='/content/drive/MyDrive/Amazon ML Challenge/amazon_ml_challenge_cleaned_dataset/amazon_ml_challenge_cleaned_dataset/train/state.json'
table = json.read_json(fn)
table

pyarrow.Table
_data_files: list<item: struct<filename: string>>
  child 0, item: struct<filename: string>
      child 0, filename: string
_fingerprint: string
_format_columns: null
_format_kwargs: struct<>
_format_type: null
_output_all_columns: bool
_split: null
----
_data_files: [[    -- is_valid: all not null
    -- child 0 type: string
["data-00000-of-00001.arrow"]]]
_fingerprint: [["ab0df949184ea6e4"]]
_format_columns: [1 nulls]
_format_kwargs: [
  -- is_valid: all not null]
_format_type: [1 nulls]
_output_all_columns: [[false]]
_split: [1 nulls]

In [ ]:
import pyarrow.parquet as pq

table = pq.read_table('/content/drive/MyDrive/Amazon ML Challenge/amazon_ml_challenge_cleaned_dataset/amazon_ml_challenge_cleaned_dataset/train/data-00000-of-00001.arrow')

ArrowInvalid: ignored

In [ ]:
import re
import pandas as pd
from pandarallel import pandarallel
from sklearn.preprocessing import MinMaxScaler

pandarallel.initialize(progress_bar=False, nb_workers=64)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
!pip install pandarallel 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16677 sha256=3c5e8cf9066601a1a75041100f68339a5e297d20b510389c79aae23933cbc5b5
  Stored in directory: /root/.cache/pip/wheels/41/01/29/deaa71fe596f8d857e57c4fb388db8861e23e6ed0b03204dcb
Successfully built pandarallel


In [ ]:
# Define a function to remove URLs, emojis, and emoticons from a given text
def _clean_text(text):
    # Handle cases where the input is None or NaN
    if text is None or pd.isna(text):
        return ''
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Define the set of allowed characters (ASCII characters plus the specified exceptions).
    allowed_chars = r'a-zA-Z0-9+\-_=:,. '

    # Use a regular expression to remove any character that is not in the allowed set.
    cleaned_text = re.sub(f'[^{allowed_chars}]', '', text)
    
    return cleaned_text

# Define a function to check if the number of digits is greater than the number of alphabetic characters
def _more_numbers_than_alphabets(row):
    text = ' '.join(row)
    num_digits = sum(char.isdigit() for char in text)
    num_alpha = sum(char.isalpha() for char in text)
    return num_digits > num_alpha

def sanitise_data(df):
    df['TITLE'] = df['TITLE'].parallel_apply(_clean_text)
    df['BULLET_POINTS'] = df['BULLET_POINTS'].parallel_apply(_clean_text)
    df['DESCRIPTION'] = df['DESCRIPTION'].parallel_apply(_clean_text)

    # Filter out rows where the number of digits is greater than the number of alphabetic characters
    df = df[~df[['TITLE', 'BULLET_POINTS', 'DESCRIPTION']].parallel_apply(_more_numbers_than_alphabets, axis=1)]

    # scaler = MinMaxScaler()
    # df[["PRODUCT_LENGTH"]] = scaler.fit_transform(df[["PRODUCT_LENGTH"]])
    return df
def sample_data(df, n):
    new_df = df.sample(n=n, random_state=42)
    new_df.reset_index(drop=True, inplace=True)
    return new_df

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Amazon ML Challenge/dataset/train.csv')
sampled_dataset = sample_data(dataset, 800_000)
santised_dataset = sanitise_data(sampled_dataset)

In [ ]:
santised_dataset.to_csv('data.csv')

In [ ]:
!cp data.csv "drive/My Drive/"